# Corretor Ortográfico em Python: aplicando técnicas de NLP

## 01. Explorando um projeto de NLP

### 04. Importando um corpus textual

In [4]:
with open("dados/artigos.txt") as f:
    artigos = f.read()

artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

### 05. Tokenização

In [5]:
texto_exemplo = "Olá, tudo bem?"
tokens = texto_exemplo.split(" ")
print(len(tokens))
print(tokens)

3
['Olá,', 'tudo', 'bem?']


## 02. Utilizando NLTK para tokenizar um texto

### 02. Refinando a tokenização

[Natural Language Toolkit - NLTK](https://www.nltk.org/)

In [16]:
import nltk

lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_tokens[:50]

['imagem',
 'Temos',
 'a',
 'seguinte',
 'classe',
 'que',
 'representa',
 'um',
 'usuário',
 'no',
 'nosso',
 'sistema',
 ':',
 'java',
 'Para',
 'salvar',
 'um',
 'novo',
 'usuário',
 ',',
 'várias',
 'validações',
 'são',
 'feitas',
 ',',
 'como',
 'por',
 'exemplo',
 ':',
 'Ver',
 'se',
 'o',
 'nome',
 'só',
 'contém',
 'letras',
 ',',
 '[',
 '*',
 '*',
 'o',
 'CPF',
 'só',
 'números',
 '*',
 '*',
 ']',
 'e',
 'ver',
 'se']

### 04. Separando palavras de tokens

In [11]:
from typing import List

def separa_palavras(lista_tokens: List[str]) -> List[str]:
    lista_palavras = list()
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

### 05. Contando palavras do Corpus

In [17]:
lista_palavras = separa_palavras(lista_tokens)
len(lista_palavras)

403104

### 06. Normalização

In [18]:
def normalizacao(lista_palavras: List[str]) -> List[str]:
    lista_normalizada = list()
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

In [19]:
lista_normalizada = normalizacao(lista_palavras)
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

### 07. Tipos de palavras

In [22]:
print(len(set(lista_normalizada)))

18465


## 03. Desenvolvendo e testando o corretor

### 03. Fatiando strings

In [33]:
palavra_exemplo = "Lgica"
def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    print(fatias)
    # palavras_geradas = insere_letras(fatias)

In [25]:
gerador_palavras(palavra_exemplo)

### 04. Operação de inserção

In [35]:
from typing import Tuple

def insere_letras(fatias: List[Tuple]) -> List:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

In [59]:
palavra_exemplo = "Lgica"

def insere_letras(fatias: List[Tuple]) -> List:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

In [74]:
gerador_palavras(palavra_exemplo)

['aLgica',
 'bLgica',
 'cLgica',
 'dLgica',
 'eLgica',
 'fLgica',
 'gLgica',
 'hLgica',
 'iLgica',
 'jLgica',
 'kLgica',
 'lLgica',
 'mLgica',
 'nLgica',
 'oLgica',
 'pLgica',
 'qLgica',
 'rLgica',
 'sLgica',
 'tLgica',
 'uLgica',
 'vLgica',
 'wLgica',
 'xLgica',
 'yLgica',
 'zLgica',
 'áLgica',
 'âLgica',
 'àLgica',
 'ãLgica',
 'éLgica',
 'êLgica',
 'èLgica',
 'ẽLgica',
 'íLgica',
 'îLgica',
 'ìLgica',
 'ĩLgica',
 'óLgica',
 'ôLgica',
 'òLgica',
 'õLgica',
 'úLgica',
 'ûLgica',
 'ùLgica',
 'ũLgica',
 'çLgica',
 'Lagica',
 'Lbgica',
 'Lcgica',
 'Ldgica',
 'Legica',
 'Lfgica',
 'Lggica',
 'Lhgica',
 'Ligica',
 'Ljgica',
 'Lkgica',
 'Llgica',
 'Lmgica',
 'Lngica',
 'Logica',
 'Lpgica',
 'Lqgica',
 'Lrgica',
 'Lsgica',
 'Ltgica',
 'Lugica',
 'Lvgica',
 'Lwgica',
 'Lxgica',
 'Lygica',
 'Lzgica',
 'Lágica',
 'Lâgica',
 'Làgica',
 'Lãgica',
 'Légica',
 'Lêgica',
 'Lègica',
 'Lẽgica',
 'Lígica',
 'Lîgica',
 'Lìgica',
 'Lĩgica',
 'Lógica',
 'Lôgica',
 'Lògica',
 'Lõgica',
 'Lúgica',
 'Lûgica',

### 06. Construindo a função corretor

In [40]:
def corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

### 07. Probabilidade das palavras geradas

In [41]:
frequencia = nltk.FreqDist(lista_normalizada)
frequencia.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6368),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [66]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)

def probabilidade(palavra_gerada: str):
    print(frequencia[palavra_gerada], total_palavras, frequencia[palavra_gerada]/total_palavras)
    return frequencia[palavra_gerada]/total_palavras

In [61]:
def corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

FreqDist({'de': 15502, 'o': 14056, 'que': 12230, 'a': 11099, 'e': 10501, 'para': 7710, 'um': 6368, 'é': 5899, 'uma': 5220, 'do': 5124, ...})